<a href="https://colab.research.google.com/github/zasuo-michikusa/Google_Colab_StableDiffusion/blob/main/StableDiffusion_AnimateDiff_custom_ipynb_20240103.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

★事前準備★

ご自分の「Googleドライブ」の「MyDrive」に 以下のフォルダ構成と

任意でGoogleColab上の「Stable-Diffusion_WEB-UI」で使用したいデータを格納してください。

MyDrive

└StableDiffusion

 　└checkpoints　※使用したいモデルデータを格納

 　└controlnet　※ツール用モデルを格納

 　└embeddings　※EasyNegativeなどを格納

 　└lora　※使用したいLoRAデータを格納

 　└VAE　※使用したいVAEデータを格納

 　└outputs　※生成データのコピー用受け皿フォルダ（フォルダのみでOK）

★このノートブックの使い方★

～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～

①まずメニュー項目の「ランタイム」から
 → ランタイムの変更（いずれかのGPUタイプを選択）

②【01】の実行ボタン（▶）を押す、

（▶）の左に✅が付いたら次の（▶）を順番に押していく

※または「ランタイム」から「すべてのセルを実行」を実行する

③【06】の処理が進み、 https://************.gradio.live
と表示されたら、そのURLをクリック

④WEB-UIを停止するときは【05】の（▶）を押して処理をクルクルを停止

⑤再度WEB-UIを使用する際は、【05】の（▶）を押し③と同様

⑥生成したデータを保存する場合は【06】の（▶）を実行

～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～


以下、「StableDiffusion_AnimateDiff環境」の起動手順です。

👇【01】Googleドライブのデータをこのノートブックへマウント（接続）

※Googleドライブにある、checkpoint、LoRA、VAE、Embedding、ControlNet用のツールモデル等を参照できるようにする

↓の（▶）を押し、出現するポップアップウィンドウから

　 Googleドライブに接続 → 「アカウントの選択」 → 「アクセスを許可」

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


👇【02】WEB-UIと各種_拡張機能のクローン（起動前にあらかじめ導入）※お好みで増減可能

In [ ]:
# Stable Diffusion Web-UIのクローン
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui.git
%cd stable-diffusion-webui

# Control Netのクローン
%cd /content/stable-diffusion-webui/extensions
!git clone https://github.com/Mikubill/sd-webui-controlnet.git
%cd /content/stable-diffusion-webui

# AnimateDiffのクローン
%cd /content/stable-diffusion-webui/extensions
!git clone https://github.com/continue-revolution/sd-webui-animatediff.git
%cd /content/stable-diffusion-webui

# Deforumのクローン
%cd /content/stable-diffusion-webui/extensions
!git clone https://github.com/deforum-art/sd-webui-deforum.git
%cd /content/stable-diffusion-webui

# Styles-Editorのクローン
%cd /content/stable-diffusion-webui/extensions
!git clone https://github.com/chrisgoringe/Styles-Editor.git
%cd /content/stable-diffusion-webui

# adetailerのクローン
%cd /content/stable-diffusion-webui/extensions
!git clone https://github.com/Bing-su/adetailer.git
%cd /content/stable-diffusion-webui

# aspect-ratio-helperのクローン
%cd /content/stable-diffusion-webui/extensions
!git clone https://github.com/thomasasfk/sd-webui-aspect-ratio-helper.git
%cd /content/stable-diffusion-webui

# prompt-all-in-oneのクローン
%cd /content/stable-diffusion-webui/extensions
!git clone https://github.com/Physton/sd-webui-prompt-all-in-one.git
%cd /content/stable-diffusion-webui

Cloning into 'stable-diffusion-webui'...
remote: Enumerating objects: 30459, done.
remote: Counting objects: 100% (148/148), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 30459 (delta 84), reused 111 (delta 67), pack-reused 30311
Receiving objects: 100% (30459/30459), 33.39 MiB | 18.69 MiB/s, done.
Resolving deltas: 100% (21311/21311), done.
/content/stable-diffusion-webui
/content/stable-diffusion-webui/extensions
Cloning into 'sd-webui-controlnet'...
remote: Enumerating objects: 8792, done.
remote: Counting objects: 100% (1071/1071), done.
remote: Compressing objects: 100% (311/311), done.
remote: Total 8792 (delta 865), reused 889 (delta 750), pack-reused 7721
Receiving objects: 100% (8792/8792), 17.29 MiB | 13.97 MiB/s, done.
Resolving deltas: 100% (5164/5164), done.
/content/stable-diffusion-webui
/content/stable-diffusion-webui/extensions
Cloning into 'sd-webui-animatediff'...
remote: Enumerating objects: 921, done.
remote: Counting objects: 100% (655/655),

👇【03】MP4形式の出力用に「ffpeg」をインストール

In [ ]:
%cd /content/stable-diffusion-webui/
!venv\Scripts\activate
!pip install imageio[ffmpeg]

/content/stable-diffusion-webui
/bin/bash: line 1: venvScriptsactivate: command not found


👇【04】Googleドライブから、「AnimateDiff」用の「MotionModel」をHugingFaceからDL

In [ ]:
%cd /content/stable-diffusion-webui/

!wget https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd14_v1.safetensors --directory-prefix=/content/stable-diffusion-webui/extensions/sd-webui-animatediff/model

!wget https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd15_v1.safetensors --directory-prefix=/content/stable-diffusion-webui/extensions/sd-webui-animatediff/model

!wget https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd15_v2.safetensors --directory-prefix=/content/stable-diffusion-webui/extensions/sd-webui-animatediff/model

!wget https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd15_v3.safetensors --directory-prefix=/content/stable-diffusion-webui/extensions/sd-webui-animatediff/model

👇【05】WEB-UIの起動（各種大容量データはGoogleドライブを参照する設定） ※起動オプション追記もお好みで可能

★ ↓の（▶）で WEB-UIの「起動」「停止」「再起動」を操作します。

In [9]:
!python launch.py --share --no-half-vae --disable-nan-check\
  --ckpt-dir "/content/drive/MyDrive/StableDiffusion/checkpoints" \
  --embeddings-dir "/content/drive/MyDrive/StableDiffusion/embeddings" \
  --lora-dir "/content/drive/MyDrive/StableDiffusion/lora" \
  --vae-dir "/content/drive/MyDrive/StableDiffusion/vae" \
  --controlnet-dir "/content/drive/MyDrive/StableDiffusion/controlnet"

Python 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
Version: v1.7.0
Commit hash: cf2772fab0af5573da775e7437e6acdca424f26e
ControlNet init warning: Unable to install insightface automatically. Please try run `pip install insightface` manually.
Launching Web UI with arguments: --share --no-half-vae --disable-nan-check --ckpt-dir /content/drive/MyDrive/StableDiffusion/checkpoints --embeddings-dir /content/drive/MyDrive/StableDiffusion/embeddings --lora-dir /content/drive/MyDrive/StableDiffusion/lora --vae-dir /content/drive/MyDrive/StableDiffusion/vae --controlnet-dir /content/drive/MyDrive/StableDiffusion/controlnet
2024-01-13 03:13:06.426444: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-13 03:13:06.426492: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for p

👇【06】いったん【05】の処理を停止させた後に（▶）を実行してください。

生成したデータ「Outputデータ」をGoogleドライブへコピーします。

≪注意≫フォルダごと追加するので、フォルダ構成が多重化していきます。

※こまめにGoogleドライブ内の「outputs」フォルダのデータをローカルに移動と削除をしてください。

In [10]:
!cp -r "/content/stable-diffusion-webui/outputs/txt2img-grids" "/content/drive/MyDrive/StableDiffusion/outputs/txt2img-grids"
!cp -r "/content/stable-diffusion-webui/outputs/txt2img-images" "/content/drive/MyDrive/StableDiffusion/outputs/txt2img-images"